In [3]:
import os
import json
import pandas as pd
import numpy as np
import tqdm
%pylab inline

DATA_PATH = './data/'

Populating the interactive namespace from numpy and matplotlib


In [5]:
with open(os.path.join(DATA_PATH, 'test_users.json'), 'r') as f:
    test_users = set(json.load(f)['users'])

In [3]:
%%time
ratings = pd.read_csv(
    os.path.join(DATA_PATH, 'ratings.csv'),
    dtype={
        'element_uid': np.uint16,
        'user_uid': np.uint32,
        'ts': np.float64,
        'rating': np.uint8
    }
)

CPU times: user 448 ms, sys: 27 ms, total: 475 ms
Wall time: 474 ms


In [4]:
%%time
transactions = pd.read_csv(
    os.path.join(DATA_PATH, 'transactions.csv'),
    dtype={
        'element_uid': np.uint16,
        'user_uid': np.uint32,
        'consumption_mode': 'category',
        'ts': np.float64,
        'watched_time': np.uint64,
        'device_type': np.uint8,
        'device_manufacturer': np.uint8
    }
)

CPU times: user 7.87 s, sys: 451 ms, total: 8.32 s
Wall time: 9.56 s


In [6]:
transactions[transactions.user_uid.isin(test_users)].user_uid.nunique()

49992

In [5]:
top500_ratings = list(ratings.groupby('element_uid').size().sort_values(ascending = False)[:500].index)

In [6]:
top500_transactions = list(transactions.groupby('element_uid').size().sort_values(ascending = False)[:500].index)

In [7]:
from collections import defaultdict

filtered_elements = defaultdict(set)

for user_uid, element_uid in tqdm.tqdm(transactions.loc[:, ['user_uid', 'element_uid']].values):
    if user_uid not in test_users:
        continue
    filtered_elements[user_uid].add(element_uid)

100%|██████████| 9643012/9643012 [01:03<00:00, 152513.86it/s]


In [8]:
result = {}
for user_uid in tqdm.tqdm(test_users):
    #result[user_uid] = [x for x in top500_ratings if x not in filtered_elements[user_uid]][:20]
    result[user_uid] = [x for x in top500_transactions if x not in filtered_elements[user_uid]][:20]

100%|██████████| 50000/50000 [03:01<00:00, 275.36it/s]


In [9]:
#with open('top500_ratings_filtered.json', 'w') as f:
with open('top500_transactions_filtered.json', 'w') as f:
    json.dump(result, f)